In [ ]:
import os
import argparse
import timeit
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Network Parameters
img_size = 64
n_channel = 3
mask_size = 25
# pathway: eye_left and eye_right
conv1_eye_size = 11
conv1_eye_out = 96
pool_eye_size = 2
pool_eye_stride = 2
conv2_eye_size = 5
conv2_eye_out = 256
conv3_eye_size = 3
conv3_eye_out = 384
conv4_eye_size = 1
conv4_eye_out = 64
eye_size = 2 * 2 * 2 * conv4_eye_out
# pathway: face
conv1_face_size = 11
conv1_face_out = 96
pool_face_size = 2
pool_face_stride = 2
conv2_face_size = 5
conv2_face_out = 256
conv3_face_size = 3
conv3_face_out = 384
conv4_face_size = 1
conv4_face_out = 64
face_size = 2 * 2 * conv4_face_out
# fc layer
fc_eye_size = 128
fc_face_size = 128
fc_face_mask_size = 256
face_face_mask_size = 128
fc_size = 128
fc2_size = 2

c32 = 32
c25 = 25

In [ ]:
eye_left_input = keras.Input(shape=(img_size, img_size, n_channel), name='eye_left')
eye_right_input = keras.Input(shape=(img_size, img_size, n_channel), name='eye_right')
face_input = keras.Input(shape=(img_size, img_size, n_channel), name='face')
face_mask_input = keras.Input(shape=(c25, c25), name='face_mask')
#y = keras.layers.Input(shape=(2), name='pos')

initializer = keras.initializers.RandomUniform(minval=0.05, maxval=0.15)
eye1 = keras.layers.Conv2D(filters=96,kernel_size=11,strides=(1,1), padding="valid", activation="relu", use_bias=True, bias_initializer=initializer)
eye2 = keras.layers.Conv2D(filters=256,kernel_size=5,strides=(1,1), padding="valid", activation="relu", use_bias=True, bias_initializer=initializer)
eye3 = keras.layers.Conv2D(filters=384,kernel_size=3,strides=(1,1), padding="valid", activation="relu", use_bias=True, bias_initializer=initializer)
eye4 = keras.layers.Conv2D(filters=64,kernel_size=1,strides=(1,1), padding="valid", activation="relu", use_bias=True, bias_initializer=initializer)

eye_left1 = eye1(eye_left_input)
eye_left1 = keras.layers.MaxPooling2D(pool_size=(2,2),strides=None, padding="valid")(eye_left1)
eye_left2 = eye2(eye_left1)
eye_left2 = keras.layers.MaxPooling2D(pool_size=(2,2),strides=None, padding="valid")(eye_left2)
eye_left3 = eye3(eye_left2)
eye_left3 = keras.layers.MaxPooling2D(pool_size=(2,2),strides=None, padding="valid")(eye_left3)
eye_left4 = eye4(eye_left3)
eye_left4 = keras.layers.MaxPooling2D(pool_size=(2,2),strides=None, padding="valid")(eye_left4)
eye_left_out = keras.layers.Flatten()(eye_left4)
	
#shouldn't use
eye_right1 = eye1(eye_right_input)
eye_right1 = keras.layers.MaxPooling2D(pool_size=(2,2),strides=None, padding="valid")(eye_right1)
eye_right2 = eye2(eye_right1)
eye_right2 = keras.layers.MaxPooling2D(pool_size=(2,2),strides=None, padding="valid")(eye_right2)
eye_right3 = eye3(eye_right2)
eye_right3 = keras.layers.MaxPooling2D(pool_size=(2,2),strides=None, padding="valid")(eye_right3)
eye_right4 = eye4(eye_right3)
eye_right4 = keras.layers.MaxPooling2D(pool_size=(2,2),strides=None, padding="valid")(eye_right4)
eye_right_out = keras.layers.Flatten()(eye_right4)

face1 = keras.layers.Conv2D(filters=96,kernel_size=11,strides=(1,1), padding="valid", activation="relu", use_bias=True, bias_initializer=initializer)(face_input)
face1 = keras.layers.MaxPooling2D(pool_size=(2,2),strides=None, padding="valid")(face1)
face2 = keras.layers.Conv2D(filters=256,kernel_size=5,strides=(1,1), padding="valid", activation="relu", use_bias=True, bias_initializer=initializer)(face1)
face2 = keras.layers.MaxPooling2D(pool_size=(2,2),strides=None, padding="valid")(face2)
face3 = keras.layers.Conv2D(filters=384,kernel_size=3,strides=(1,1), padding="valid", activation="relu", use_bias=True, bias_initializer=initializer)(face2)
face3 = keras.layers.MaxPooling2D(pool_size=(2,2),strides=None, padding="valid")(face3)
face4 = keras.layers.Conv2D(filters=64,kernel_size=1,strides=(1,1), padding="valid", activation="relu", use_bias=True, bias_initializer=initializer)(face3)
face4 = keras.layers.MaxPooling2D(pool_size=(2,2),strides=None, padding="valid")(face4)
face_out = keras.layers.Flatten()(face4)
	
# fc layer
# eye
eyes_concat = keras.layers.Concatenate(axis=-1) ([eye_left_out, eye_right_out])
eyes_dense_out = keras.layers.Dense(128, activation="relu",use_bias=True,bias_initializer=initializer)(eyes_concat)
# face
face_dense = keras.layers.Dense(128, activation="relu",use_bias=True,bias_initializer=initializer)(face_out)
face_mask = keras.layers.Flatten()(face_mask_input)
face_mask = keras.layers.Dense(256, activation="relu",use_bias=True,bias_initializer=initializer)(face_mask)
face_concat = keras.layers.Concatenate(axis=-1) ([face_dense, face_mask])
face_dense_out = keras.layers.Dense(256, activation="relu",use_bias=True,bias_initializer=initializer)(face_concat)
# all
concat = keras.layers.Concatenate(axis=-1) ([eyes_dense_out, face_dense_out])
dense1 = keras.layers.Dense(128, activation="relu",use_bias=True,bias_initializer=initializer)(concat)
out = keras.layers.Dense(2, activation="relu", use_bias=True,bias_initializer=initializer, name='y_output')(dense1)

model = keras.Model(
    inputs = {'eye_left':eye_left_input, 
              'eye_right':eye_right_input, 
              'face':face_input, 
              'face_mask':face_mask_input},
    outputs = {'y_output':out},
)

model.compile(
    optimizer = keras.optimizers.Adam(learning_rate=0.01),
    loss = keras.losses.mean_squared_error,
)

#def train(self, train_data, val_data, lr=1e-3, batch_size=128, max_epoch=1000, min_delta=1e-4, patience=10, print_per_epoch=10, out_model='my_model'):

In [ ]:
# Import data
npzfile = np.load("/content/drive/MyDrive/Colab Notebooks/dataset/eye_tracker_train_and_val.npz")
train_eye_left = npzfile["train_eye_left"].astype(np.float32)
train_eye_right = npzfile["train_eye_right"].astype(np.float32)
train_face = npzfile["train_face"].astype(np.float32)
train_face_mask = npzfile["train_face_mask"].astype(np.float32)
train_y = npzfile["train_y"].astype(np.float32)
#val_eye_left = npzfile["val_eye_left"]
#val_eye_right = npzfile["val_eye_right"]
#val_face = npzfile["val_face"]
#val_face_mask = npzfile["val_face_mask"]
#val_y = npzfile["val_y"]

In [ ]:
for i in range(train_eye_left.shape[0]):
  train_eye_left[i] = train_eye_left[i]/255.
  train_eye_right[i] = train_eye_right[i]/255.
  train_face[i] = train_face[i]/255.

In [ ]:
max=0
min=0
for i in range(train_y.shape[0]):
  if train_y[i][0]>max:
    max=train_y[i][0]
  if train_y[i][1]>max:
    max=train_y[i][1]
  if train_y[i][0]<min:
    min=train_y[i][0]
  if train_y[i][1]<min:
    min=train_y[i][1]
amplitude = max+abs(min)
for i in range(train_y.shape[0]):
  train_y[i]=(train_y[i]+abs(min))/amplitude

In [ ]:
def gen(dim):
  i = 0
  while i<dim:
    yield {'eye_left': train_eye_left[i],
           'eye_right': train_eye_right[i],
           'face': train_face[i],
           'face_mask': train_face_mask[i]}, {'y_output': train_y[i]}
    i += 1

In [ ]:
dim = train_eye_left.shape[0]
train_dataset = tf.data.Dataset.from_generator(gen, args=[dim], output_signature=(
    {'eye_left': tf.TensorSpec(shape=(64, 64, 3), dtype=tf.float32),
    'eye_right': tf.TensorSpec(shape=(64, 64, 3), dtype=tf.float32),
    'face': tf.TensorSpec(shape=(64, 64, 3), dtype=tf.float32),
    'face_mask': tf.TensorSpec(shape=(25, 25), dtype=tf.float32)},
    {'y_output': tf.TensorSpec(shape=(2), dtype=tf.float32)}
))

In [ ]:
model.fit(
    train_dataset.batch(10),
    epochs=10,
)

Epoch 1/10
4800/4800 [==============================] - 351s 71ms/step - loss: 1792.6218
Epoch 2/10
 694/4800 [===>..........................] - ETA: 4:49 - loss: 0.2324

In [ ]:
ab=1
#train_dataset.take(1)
for ab,bc in train_dataset.repeat().batch(1).take(1):
  print(bc)
#for i,ab, bc in gen():
 # print(i)
  #if i>10:
   # break

In [ ]:
print ("train_eye_left.shape")
print(train_eye_left.shape)
print ("train_eye_right.shape")
print(train_eye_right.shape)
print ("train_face.shape")
print(train_face.shape)
print ("train_face_mask.shape")
print(train_face_mask.shape)
print ("train_y.shape")
print(train_y.shape)

In [ ]:
def normalize(data):
    data = data/255. # scaling
    data = data - np.mean(data, axis=0) # normalizing
    return data

#train_dataset = tf.data.Dataset.from_tensor_slices((train_eye_left,train_eye_right,train_face, train_face_mask, train_y))
#val_dataset = tf.data.Dataset.from_tensor_slices((val_eye_left,val_eye_right,val_face, val_face_mask, val_y))

def prepare_data(data):
    eye_left, eye_right, face, face_mask, y = data
    eye_left = normalize(eye_left)
    print ('eye_left normalized')
    eye_right = normalize(eye_right)
    print ('eye_right normalized')
    face = normalize(face)
    print ('face normalized')
    face_mask = np.reshape(face_mask, (face_mask.shape[0], -1)).astype('float32')
    print ('face mask normalized')
    y = y.astype('float32')
    print ('y normalized')
    return [eye_left, eye_right, face, face_mask, y]

def shuffle_data(data):
    idx = np.arange(data[0].shape[0])
    np.random.shuffle(idx)
    for i in range(len(data)):
        data[i] = data[i][idx]
    return data

def next_batch(data, batch_size):
    for i in np.arange(0, data[0].shape[0], batch_size):
        # yield a tuple of the current batched data
        yield [each[i: i + batch_size] for each in data]

In [ ]:
def load_model(session, save_path):
    """ Loads a saved TF model from a file.
    Args:
        session: The tf.Session to use.
        save_path: The save path for the saved session, returned by Saver.save().
    Returns:
        The inputs placehoder and the prediction operation.
    """
    print ("Loading model from file '%s'... % save_path ")

    meta_file = save_path + ".meta"
    if not os.path.exists(meta_file):
        raise Exception("ERROR: Expected .meta file '%s', but could not find it." % meta_file)

    saver = tf.train.import_meta_graph(meta_file)
    # It's finicky about the save path.
    save_path = os.path.join("./", save_path)
    saver.restore(session, save_path)

    # Check that we have the handles we expected.
    return extract_validation_handles(session)